In [1]:
import numpy as np
import pandas as pd
import keras
import random
import tensorflow as tf
from sklearn import preprocessing
from sklearn import metrics
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib.pyplot as plt
import os, os.path, sys
import copy

In [3]:
#random.seed(7)
#np.random.seed(7)
#tf.random.set_seed(7)
keras.utils.set_random_seed(7)

In [40]:
path = r"C:\Users\mata2\Desktop\master\podaci\10k\Y"

In [41]:
dir = os.listdir(path)

In [42]:
file_names = []
for file in dir:
    file_names.append(file)

In [43]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    if current_file[0][i] < min_kord[i]:
      min_kord[i] = current_file[0][i]
    elif current_file[0][i] > max_kord[i]:
      max_kord[i] = current_file[0][i]

In [8]:
kord_sum = [0 for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    kord_sum[i] += current_file[0][i]

napravljene su prosecne koordinate

In [44]:
average_kord = [x//len(file_names) for x in kord_sum]

In [45]:
synth_path = r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\sinteticki podaci\10k\Y"

In [46]:
dir2 = os.listdir(synth_path)

In [47]:
file_names_synt = []
for file in dir2:
    file_names_synt.append(file)

In [49]:
train_synt = np.concatenate([pd.read_csv(synth_path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names_synt], axis=0)
test_synt = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names], axis=0)

best step = 1

In [50]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [51]:
time_steps = 1

In [52]:
train_X, train_y = create_timeseries_split(train_synt, time_steps)

In [53]:
test_X, test_y = create_timeseries_split(test_synt, time_steps)

In [54]:
input_shape = (time_steps,1)

In [58]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
modelMSE = Sequential()
modelMSE.add(GRU(units = 32, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
#modelMSE.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)


In [59]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 1024, verbose = 2)

Epoch 1/10
59571/59571 - 184s - loss: 56850192.0000 - 184s/epoch - 3ms/step
Epoch 2/10
59571/59571 - 188s - loss: 1472.8597 - 188s/epoch - 3ms/step
Epoch 3/10
59571/59571 - 179s - loss: 1471.9635 - 179s/epoch - 3ms/step
Epoch 4/10
59571/59571 - 174s - loss: 1472.0907 - 174s/epoch - 3ms/step
Epoch 5/10
59571/59571 - 172s - loss: 1471.4720 - 172s/epoch - 3ms/step
Epoch 6/10
59571/59571 - 171s - loss: 1471.0583 - 171s/epoch - 3ms/step
Epoch 7/10
59571/59571 - 171s - loss: 1470.9543 - 171s/epoch - 3ms/step
Epoch 8/10
59571/59571 - 171s - loss: 1469.8052 - 171s/epoch - 3ms/step
Epoch 9/10
59571/59571 - 171s - loss: 1471.0563 - 171s/epoch - 3ms/step
Epoch 10/10
59571/59571 - 171s - loss: 1469.3071 - 171s/epoch - 3ms/step


In [60]:
train_scores = modelMSE.evaluate(test_X, test_y)

57188/57188 [==============================] - 61s 1ms/step - loss: 1338.5841


In [61]:
test_predict_MSE = modelMSE.predict(test_X)

57188/57188 [==============================] - 59s 1ms/step


In [74]:
uspeli_indeksiMSE = []
num_correct_MSE = 0

for i in range(61000):
    if(min_kord[i] <= int(np.round(test_predict_MSE[i][0],0)) <= max_kord[i]):
        num_correct_MSE += 1
        uspeli_indeksiMSE.append(i)

print("Procenat koliko MSE upada u pojas", num_correct_MSE/61000)
print("Koliko je MSE upalo u pojas", num_correct_MSE)

Procenat koliko MSE upada u pojas 0.9189836065573771
Koliko je MSE upalo u pojas 56058


In [71]:
for i in range(61000):
    if not(min_kord[i] <= int(np.round(test_predict_MSE[i][0])) <= max_kord[i]):
        print(min_kord[i], int(np.round(test_predict_MSE[i][0])), max_kord[i])
    

0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 2
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 0
0 -1 1
0 -1 1
0 -1 0
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
-1 -2 0
-1 -2 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
-1 -2 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
0 -1 1
29846 29854 29853
66494 66507 66505
66544 66562 66556
66595 66614 66607
66645 66662 66659
67342 67356 67355
68245 68257 68256
68293 68308 68307
144245 144244 144258
144496 144495 144509
144745 144744 144759
145647 145646 145664
148744 148743 148760
148993 149006 149004
149043 149062 149058
149095 149114 149110
149145 149163 149162
152295 152307 152306
152396 152406 152405
153343 153356 153355
153743 153758 153756
153793 153807 153806
163996 163995 164006
164047 164046 164062
164247 164246 164258
164298 164297 164307
164748 164747 164761
165748 165747 165759

174828 174827 174839
174778 174777 174785
174729 174728 174736
154981 154980 154995
154875 154874 154896
154084 154083 154095
154034 154033 154045
153984 153983 153994
152985 152984 152995
152487 152486 152497
150780 150779 150796
150731 150730 150747
150233 150232 150246
133880 133894 133893
133379 133394 133393
132633 132644 132643
131635 131646 131645
130636 130646 130645
129881 129895 129894
129686 129697 129696
129637 129646 129645
129435 129447 129446
129381 129396 129394
129135 129145 129144
128880 128894 128893
128829 128843 128842
128780 128794 128793
128734 128745 128744
128684 128696 128695
128635 128646 128643
128586 128594 128592
128085 128092 128091
128035 128043 128042
127933 127945 127943
127880 127893 127892
127635 127644 127643
127381 127394 127392
127135 127145 127143
127085 127093 127092
126880 126894 126893
126637 126645 126644
125637 125647 125646
125587 125596 125595
125136 125147 125145
124686 124696 124695
124635 124646 124645
123635 123646 123645
123185 123196

In [73]:
modelMSE.save(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\modeli\10k\Y")

INFO:tensorflow:Assets written to: C:\Users\mata2\Desktop\master\Axis-projection-RNN\modeli\10k\Y\assets


INFO:tensorflow:Assets written to: C:\Users\mata2\Desktop\master\Axis-projection-RNN\modeli\10k\Y\assets
